In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

### mix two dataset, rename ids of the 2nd dataset

In [2]:
dataset1_path = "/home/ns1254/dataset_mimicgen/coffee_gfs_109_2_sub.hdf5"

In [3]:
f1 = h5py.File(dataset1_path, "r")
demos = list(f1["data"].keys())

In [4]:
lengths=[]
for demo_name in demos:
    demo=f1['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  40
Max length:  777
Min length:  516
Mean length:  643.65


In [5]:
for k in f1['mask'].keys():
    ks=f1['mask'][k]
    print(k, ks)

g20 <HDF5 dataset "g20": shape (20,), type "|S8">
g40 <HDF5 dataset "g40": shape (40,), type "|S8">


In [6]:
f1.close()

### load 2nd dataset

In [7]:
dataset2_path = "/home/ns1254/dataset_mimicgen/lay_coffee_59.hdf5" 
f2 = h5py.File(dataset2_path, "r")
demos2 = list(f2["data"].keys())

lengths=[]
for demo_name in demos2:
    demo=f2['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos2))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  59
Max length:  4000
Min length:  400
Mean length:  1857.8983050847457


In [8]:
for k in f2['mask'].keys():
    ks=f2['mask'][k]
    print(k, ks.shape)

sg15 (15,)
train (54,)
valid (5,)


In [9]:
demos2=[demo.decode('utf-8') for demo in f2['mask']['sg15']]
# demos3=[demo.decode('utf-8') for demo in f2['mask']['more22']]
# demos2=demos2[-30:]
len(demos2) 

15

In [10]:
demos2copy=demos2 
demos2copy

['demo_12',
 'demo_14',
 'demo_17',
 'demo_19',
 'demo_23',
 'demo_28',
 'demo_3',
 'demo_32',
 'demo_38',
 'demo_39',
 'demo_44',
 'demo_45',
 'demo_46',
 'demo_5',
 'demo_52']

In [11]:
len(demos2copy)

15

### now open mix dataset for modifying

In [12]:
dataset1_path 

'/home/ns1254/dataset_mimicgen/coffee_gfs_109_2_sub.hdf5'

In [13]:
f1 = h5py.File(dataset1_path, "r+")
demos = list(f1["data"].keys())
len(demos)

40

In [14]:
max_id=np.max([int(demo.split('_')[1]) for demo in demos])
print(f'max_id: {max_id}')

next_id=max_id+1
print(f'next_id: {next_id}')

max_id: 39
next_id: 40


In [15]:
# demos2copy=demos2.copy()

In [16]:
i=next_id
new_demos=[]
for demo_name in tqdm.tqdm(demos2copy):
    demo2=f2['data'][demo_name]
    num_samples= demo2.attrs['num_samples']
    model_file= demo2.attrs['model_file']

    bdemo_name="demo_"+str(i)
    new_demos.append(bdemo_name)
    i = i+1

    f1["data"].create_group(bdemo_name)


    ks=['actions', 'dones', 'rewards', 'states']
    for k in ks:
        kdata = demo2[k]
        f1["data"][bdemo_name].create_dataset(k, data=kdata)

    obs_keys =demo2['obs'].keys()
    next_obs_keys= demo['next_obs'].keys()
    for k in obs_keys:
        kdata = demo2['obs'][k]
        f1["data"][bdemo_name].create_dataset(f'obs/{k}', data=kdata)
    for k in next_obs_keys:
        kdata = demo2['next_obs'][k]
        f1["data"][bdemo_name].create_dataset(f'next_obs/{k}', data=kdata)

    f1["data"][bdemo_name].attrs["num_samples"] = num_samples
    f1["data"][bdemo_name].attrs["model_file"] = model_file

# f.close()
f2.close()

100%|██████████| 15/15 [00:01<00:00, 11.03it/s]


In [16]:
# f.close()
# f2.close()

In [17]:
new_demos=np.array(new_demos, dtype='S8')
new_demos.shape

(15,)

In [18]:
hdf5_path=dataset1_path

filter_keys=sorted([elem for elem in new_demos])
filter_name='lay15'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [19]:
f1['mask'].keys()

<KeysViewHDF5 ['g20', 'g40', 'lay15']>

In [20]:
f1.close()